# Лабораторная работа №1. Регрессия

**Тема:** *Сравнение линейной регрессии, случайного леса (Random Forest Regressor), градиентного бустинга (XGBoost) на задаче регрессии.*

**Цели:**

- *Научиться строить, оптимизировать и оценивать регрессионные модели.*
- *Понять, как интерпретировать важность признаков (feature importance).*
- *Понять, как увеличивать кол-во признаков (feature tuning).*
- *Исследовать влияние преобразований признаков (фичей), регуляризации и гиперпараметров.*

**Условия:**

- *numpy, pandas, scikit-learn, XGBoost*
- *выбрать датасет для 1-ой и 2-ой лабораторной работы*

**Пункты:**

1. Подготовка данных + Feature Tuning
2. Создание метрик
3. Базовая модель линейной регрессии
4. Улучшение линейной регрессии
5. Случайный лес регрессор
6. Градиентный бустинг (XGBoost)
7. Feature Importance
8. Написание своих реализаций (классы)
9. Подведение итогов


## О команде (ЗАПОЛНИТЬ СВОИМИ ДАННЫМИ)

| Фамилия И.О.        | Группа | Роль в команде | Что делал                    |
|---------------------|--------|----------------|------------------------------|
| Солодова С. М.         | 309    |        |                       |
| Штыхно И.          | 309    |        |                       |
| Мазепа И.         | 309    |        |                       |

 Датасет: [student habits vs academic perfomance](https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance)

## Описание датасета:

Датасет Student Habits & Performance представляет собой комплексное исследование факторов, влияющих на академическую успеваемость студентов. Он содержит данные о 1000 студентах и включает 16 различных характеристик, охватывающих демографические показатели, учебные привычки, образ жизни и социально-экономические факторы. Целевой переменной является exam_score — экзаменационные баллы студентов, которые варьируются от 18.4 до 100.0 баллов со средним значением 69.6 балла.

Датасет структурирован по четырем основным блокам признаков: демографические данные (возраст, пол), академические факторы (часы учебы в день, процент посещаемости, уровень образования родителей), характеристики образа жизни (время в социальных сетях, просмотр Netflix, часы сна, качество питания, частота упражнений) и факторы окружающей среды (подработка, качество интернета, оценка психического здоровья, участие во внеучебной деятельности). Это делает датасет особенно ценным для изучения многофакторного влияния на успеваемость студентов и построения предиктивных моделей в области образовательной аналитики. Одна из проблем качества данных — пропуски в признаке уровня образования родителей, что требует предварительной обработки перед анализом.

---

### Таблица признаков

| № | Признак                      | Тип данных | Описание                                                                 |
|---|------------------------------|------------|--------------------------------------------------------------------------|
| 1 | student_id                   | object     | Уникальный идентификатор студента (S1000-S1999)                         |
| 2 | age                          | int        | Возраст студента (17-24 лет)                                            |
| 3 | gender                       | object     | Пол студента (Female, Male, Other)                                      |
| 4 | study_hours_per_day          | float      | Количество часов учебы в день (0.0-8.3)                                 |
| 5 | social_media_hours           | float      | Часы в социальных сетях в день (0.0-7.2)                                |
| 6 | netflix_hours                | float      | Часы просмотра Netflix в день (0.0-5.4)                                 |
| 7 | part_time_job                | object     | Наличие подработки (No, Yes)                                            |
| 8 | attendance_percentage        | float      | Процент посещаемости занятий (56.0-100.0%)                              |
| 9 | sleep_hours                  | float      | Количество часов сна в сутки (3.2-10.0)                                 |
| 10| diet_quality                 | object     | Качество питания (Fair, Good, Poor)                                     |
| 11| exercise_frequency           | int        | Частота физических упражнений в неделю (0-6)                            |
| 12| parental_education_level     | object     | Уровень образования родителей (Master, High School, Bachelor) - 9.1% пропусков |
| 13| internet_quality             | object     | Качество интернет-соединения (Average, Poor, Good)                      |
| 14| mental_health_rating         | int        | Самооценка психического здоровья (шкала 1-10)                           |
| 15| extracurricular_participation| object     | Участие во внеучебной деятельности (Yes, No)                            |
| 16| exam_score                   | float      | **ЦЕЛЕВАЯ ПЕРЕМЕННАЯ** - Экзаменационный балл (18.4-100.0, среднее: 69.6) |


## 0. Глобальная настройка проекта


In [ ]:
RND_SEED = 21
USE_AUTO_POLY = True

## 1. Подготовка данных + Feature Tuning

### 1.1. Загрузка датасета

Подключим `Google Drive` и загрузим наш датасет используя `Pandas.DataFrame`

Подключение к гугл диску

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from pathlib import Path

drive_path = Path('/content/drive')
drive.mount(str(drive_path))

Загрузка датасета `pd.read_csv(path_to_dataset: str)`

In [ ]:
file_path = drive_path / 'MyDrive' / 'student_habits_performance.csv'
df = pd.read_csv(file_path)

Получение статистки по датасету, необходимую для анализа. Подумать что можно сделать и какие выводы нужны.

Посмотрим первые три строки датасета `df.head(n: int)`

In [ ]:
df.head(3)

У нас тут имеется ненужный атрибут `student_id`. Можем его удалить.

In [ ]:
if 'student_id' in df.columns:
    df = df.drop(columns=['student_id'])

### 1.2. Сбор основной информации

Получение общей информации `df.info()`

In [ ]:
df.info()

In [ ]:
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

def normalize_series(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.strip()
         .str.replace('\u00A0', ' ', regex=False)   # NBSP -> space
         .str.replace(r'\s+', ' ', regex=True)      # collapse spaces
         .str.title()                               # простая капитализация
    )

print('Уникальные значения (нормализованные):')
normalized_uniques = {}
for col in cat_cols:
    vals = normalize_series(df[col])
    normalized_uniques[col] = sorted(vals.unique().tolist())
    print(f'- {col}: {normalized_uniques[col]}')

Получение всей статистики `df.describe()`

In [ ]:
df.describe()

**Вывод:**
####**Общая характеристика данных:**
* Выборка включает 1000 студентов с полными данными (нет пропусков)

* Возраст студентов: 17-24 года (среднее 20.5 лет)

* Экзаменационные баллы варьируются от 18.4 до 100.0 (среднее 69.6)

####**Ключевые наблюдения:**

**Учебные привычки:**
* Посещаемость высокая (среднее 84.1%, минимум 56%)

* Учеба: 3.55 часа/день в среднем, но разброс большой (0-8.3 часа)

* 25% студентов учатся меньше 2.6 часов в день

**Времяпрепровождение:**
* Соцсети: 2.5 часа/день в среднем, некоторые до 7.2 часа

* Netflix: 1.8 часа/день, относительно умеренное использование

* Сон: 6.47 часа в среднем, но 25% студентов спят меньше 5.6 часа

**Здоровье и активность:**
* Физнагрузки: 3 раза/неделю в среднем, но 25% студентов почти не занимаются (≤1 раз)

* Психическое здоровье: средняя оценка 5.44/10, значительный разброс (1-10)

**Потенциальные проблемы:**
* Недостаток сна у значительной части студентов

* Низкая физическая активность у 25% выборки

* Высокий разброс учебного времени указывает на неравномерную нагрузку

Проверка на пропуски данных `df.isnull().sum()`

In [ ]:
print(df.isnull().sum())

В выводе видно, что в столбце "parental_education_level" есть пропущенные значения.

Выведим уникальные значения этой колонки, чтобы далее принять решения как с ним работать.

In [ ]:
print((df['parental_education_level']).unique())

**Вывод:** в датасетe пропуски. Заменим пустые значения модой.

**Мода** - самое часто встречающееся значение.

In [ ]:
df_processed = df.copy()
education_mode = df_processed['parental_education_level'].mode()[0]
df_processed['parental_education_level'] = df_processed['parental_education_level'].fillna(education_mode)
print(df_processed.isnull().sum())
df_processed.info()

Посмотрим на распределение целевой переменной

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))

n, bins, patches = plt.hist(df['exam_score'], bins=25, alpha=0.7,
                           color='lightblue', edgecolor='navy', linewidth=0.8)

mean_score = df['exam_score'].mean()
median_score = df['exam_score'].median()

plt.axvline(mean_score, color='red', linestyle='--', linewidth=2,
           label=f'Среднее: {mean_score:.1f}')
plt.axvline(median_score, color='green', linestyle='-', linewidth=2,
           label=f'Медиана: {median_score:.1f}')

plt.title('Распределение экзаменационных баллов студентов', fontsize=14, fontweight='bold')
plt.xlabel('Экзаменационный балл', fontsize=12)
plt.ylabel('Количество студентов', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

stats_text = f'n = {len(df)}\nСтд. откл. = {df["exam_score"].std():.1f}'
plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()


**Вывод:**

Распределение экзаменационных баллов близко к нормальному с небольшим смещением влево. Медиана (70.5) немного выше среднего (69.6), что указывает на наличие небольшой группы студентов с низкими результатами, которые снижают общее среднее значение.

Большинство студентов показывают результаты в среднем диапазоне, при этом экстремально низкие баллы встречаются реже, чем высокие, что свидетельствует о достаточно успешной академической подготовке основной массы студентов.

In [ ]:
import matplotlib.pyplot as plt

# Один вертикальный box plot
fig, ax = plt.subplots(figsize=(7, 6))

ax.boxplot(
    df['exam_score'],
    patch_artist=True,
    boxprops=dict(facecolor='lightcoral', alpha=0.7),
    medianprops=dict(color='darkred', linewidth=2)
)

ax.set_title('Ящик с усами (вертикальный)')
ax.set_ylabel('Экзаменационный балл')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()



Вывод: видим выбросы, малое количество баллов набрало значительно меньшее количество студентов. Значит, в моделях, чувствительных к выбросам, таких как линейная регрессия, нужно от них избавиться.

In [ ]:
# 3. КОДИРОВАНИЕ ПОРЯДКОВЫХ ПРИЗНАКОВ
ordinal_mappings = {
    'diet_quality': {'Poor': 1, 'Fair': 2, 'Good': 3},
    'internet_quality': {'Poor': 1, 'Average': 2, 'Good': 3},
    'parental_education_level': {'High School': 1, 'Bachelor': 2, 'Master': 3}
}

for col, mapping in ordinal_mappings.items():
    df_processed[col] = df_processed[col].map(mapping)  # Используем df_processed

# 4. ONE-HOT ENCODING ДЛЯ НОМИНАЛЬНЫХ ПРИЗНАКОВ
nominal_cols = ['gender', 'part_time_job', 'extracurricular_participation']
df_processed = pd.get_dummies(df_processed, columns=nominal_cols, prefix=nominal_cols, drop_first=True)

#print(f"Размерность матрицы: {correlation_matrix.shape}")

df_processed.info()
df_processed.head(3)

**Вывод:** мы избавились от строковых значений в наших данных.

Посмотрим на корреляцию данных.

***Определение 1: Корреляция данных — это статистическая мера, показывающая, насколько и в каком направлении связаны между собой две переменные.***

In [ ]:
import seaborn as sns

def draw_corr_matrix(df):
    plt.figure(figsize=(10,8))
    sns.heatmap(df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Корреляционная матрица признаков")
    plt.show()

draw_corr_matrix(df_processed)

**Анализ корреляционной матрицы:**

Корреляционный анализ выявил, что наиболее значимым предиктором экзаменационных баллов является количество часов учебы в день (r=0.82), демонстрируя сильную положительную связь. Умеренную положительную корреляцию с целевой переменной показывают оценка психического здоровья (r=0.31) и частота физических упражнений (r=0.15). Слабое негативное влияние на успеваемость оказывают время, проводимое в социальных сетях (r=-0.16) и просмотр Netflix (r=-0.16).

Большинство остальных признаков демонстрируют слабые корреляции с целевой переменной (|r| < 0.3), что типично для социально-поведенческих данных. Анализ не выявил критических проблем мультиколлинеарности между предикторами, за исключением ожидаемых отрицательных корреляций между dummy-переменными пола. Полученные результаты подтверждают гипотезу о доминирующем влиянии академических привычек на учебные достижения студентов по сравнению с демографическими и lifestyle-факторами.

**!!! ВАЖНО !!!**

**ЕСЛИ МЫ СОБИРАЕМСЯ УЧИТЬ ЛИНЕЙНУЮ МОДЕЛЬ, И ДАННЫЕ КОРРЕЛИРУЮТ (МУЛЬТИКОЛЛИНЕАРНОСТЬ), ТО НУЖНО ЛИБО УДАЛИТЬ ОДИН ИЗ ПРИЗНАКОВ, ЛИБО СОЗДАТЬ НОВЫЙ ПРИЗНАК НА ИХ ОСНОВЕ И ИХ УДАЛИТЬ**

### 1.3. Подготовка датасета под разные задачи

Cравнение подготовки

| Модель                  | Масштабирование | Корреляция критична | Выбросы критичны | Feature Engineering рекомендуем                  |
| ----------------------- | --------------- | ------------------- | ---------------- | ------------------------------------------------ |
| Линейная регрессия      | Да              | Да                  | Да               | Полиномы, логарифмы, отношения                   |
| Random Forest Regressor | Нет             | Нет                 | Нет              | Соотношения, интеракции                          |
| XGBoost Regressor       | Нет             | Нет                 | Нет              | Соотношения, интеракции, логарифмы (опционально) |

#### 1.3.1 Линейная регрессия (Linear Regression / Ridge / Lasso)

**Особенности модели:**

- Чувствительна к масштабу признаков и мультиколлинеарности.
- Чувствительна к выбросам.

Сделаем копию датасета

In [ ]:
df_linear = df_processed.copy()

Далее избавимся от выбросов. Удалим крайние 1% у функции распределения



In [ ]:
lower_limit = df_linear['exam_score'].quantile(0.01)
upper_limit = df_linear['exam_score'].quantile(0.99)

df_linear = df_linear[(df_linear['exam_score'] >= lower_limit) &
                 (df_linear['exam_score'] <= upper_limit)]


Убедимся что выбросов нет

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(7, 6))

ax.boxplot(
    df_linear['exam_score'],
    patch_artist=True,
    boxprops=dict(facecolor='lightcoral', alpha=0.7),
    medianprops=dict(color='darkred', linewidth=2)
)

ax.set_title('Ящик с усами')
ax.set_ylabel('Экзаменационный балл')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


Рассмотрим скошенность данных

***Определение 2: Скошенность – это мера асимметрии распределения признака***

* **Скошенность > 0 (положительная):** Хвост распределения тянется вправо (большие значения встречаются реже).

* **Скошенность < 0 (отрицательная):** Хвост распределения тянется влево (малые значения редки).

* **Скошенность ≈ 0:** Практически нормальное распределение (симметричное).


In [ ]:
skew_values = df_linear.select_dtypes(include='float').skew()
skew_values_hard = skew_values[abs(skew_values) > 0.2]  # выделяем сильноскошенные данные
print(skew_values_hard)  # сильно скошенные

skew_columns_hard = list(skew_values_hard.index)
print(f'Скошенные столбцы {skew_columns_hard}')

In [ ]:
df_processed.head(3)

In [ ]:
import math

def draw_skew(df: pd.DataFrame, n_cols=4):
    numeric_cols = df.select_dtypes(include='float').columns
    n_rows = math.ceil(len(numeric_cols) / n_cols)

    fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols*4, n_rows*3))
    axs = axs.flatten()  # делаем одномерным массивом для удобства

    for i, col in enumerate(numeric_cols):
        sns.histplot(df[col], kde=True, ax=axs[i])
        axs[i].set_title(f'{col}')

    for j in range(i+1, len(axs)):
        axs[j].set_visible(False)

    plt.tight_layout()
    plt.show()


draw_skew(df_linear)

**Анализ распределений признаков:**

Анализ гистограмм показал, что большинство ключевых признаков имеют приемлемые для регрессионного моделирования распределения. Академические признаки (study_hours_per_day, attendance_percentage, exam_score) и базовые характеристики (sleep_hours) демонстрируют нормальные или близкие к нормальным распределения, что оптимально для линейной регрессии. Lifestyle-признаки (social_media_hours, netflix_hours) показывают умеренную правостороннюю асимметрию с концентрацией малых значений, что типично для поведенческих данных. Некоторые категориальные признаки после кодирования (parental_education_level, internet_quality) имеют бимодальные распределения, указывающие на смешение групп. Демографические признаки (age, mental_health_rating) показывают относительно равномерные распределения. **Критической проблемой является только сильная правосторонняя асимметрия netflix_hours, которая может потребовать логарифмической трансформации.** В целом, качество распределений позволяет применять стандартные регрессионные алгоритмы без существенной предобработки данных.

Можно уменьшить влияние скошенности с помошью:

- Логарифмирование (Уменьшаем положительный хвост)
- Квадратный корень  (сглаживаем умеренные хвосты)
- `Box-Cox` или `Yeo-Johnson` трансформации (более гибкие)

Мы же просто прологарифмируем :)

In [ ]:
df_linear['netflix_hours'] = np.log1p(df_linear['netflix_hours'])
df_linear['attendance_percentage'] = np.log1p(df_linear['attendance_percentage'])
draw_skew(df_linear)

Посмотрим отдельно зависимости от целевого признака



In [ ]:
import seaborn as sns
target = 'exam_score'
num_cols = [
'study_hours_per_day',
'social_media_hours',
'netflix_hours',
'attendance_percentage',
'sleep_hours'
]

cols = 2
rows = int(np.ceil(len(num_cols)/cols))
plt.figure(figsize=(12, 4*rows))
for i, xcol in enumerate(num_cols, start=1):
    ax = plt.subplot(rows, cols, i)
    # Scatter + LOWESS
    sns.regplot(
        data=df, x=xcol, y=target,
        lowess=True,
        scatter_kws={'alpha':0.3, 's':18},
        line_kws={'color':'crimson', 'lw':2},
        ax=ax
    )
    # Линейная линия поверх
    sns.regplot(
        data=df, x=xcol, y=target,
        scatter=False, order=1,
        line_kws={'color':'steelblue', 'lw':1.2, 'alpha':0.7},
        ax=ax
    )
    ax.set_title(f'{target} vs {xcol}')
    ax.grid(alpha=0.25)
plt.tight_layout()
plt.show()

In [ ]:
disc_cols = []
for c in df.columns:
    if c == target:
        continue
    if pd.api.types.is_integer_dtype(df[c]):
        u = df[c].nunique()
        if 3 <= u <= 30:
            disc_cols.append(c)
print('Дискретные числовые признаки:', disc_cols)

def plot_point_ci95(df, xcol, ycol=target):
    plt.figure(figsize=(7,4))
    sns.pointplot(
        data=df, x=xcol, y=ycol,
        estimator=np.mean,
        errorbar=('ci', 95),   # доверительный интервал для среднего
        color='black'
    )
    plt.title(f'{ycol} by {xcol}: mean ± 95% CI')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

for col in disc_cols:
    plot_point_ci95(df, col, target)

Создадим навые признаки
1) Учебные часы: центр + квадрат
study_hours_c — центрированная линейная часть, снижает коллинеарность с квадратичным членом и стабилизирует коэффициенты.

study_hours_c_sq — мягкая нелинейность для эффекта убывающей отдачи на больших значениях учебных часов

При желании исходный столбец можно убрать, оставив центрированную версию как линейную компоненту.

2) Взаимодействие сна и ментального состояния
sleep_mental = sleep_hours × mental_health_rating моделирует совместный восстановительный эффект, который не улавливается простой суммой признаков.

Линейная регрессия без взаимодействия предполагает независимые наклоны; этот признак позволяет наклону одного фактора зависеть от уровня другого.

3) Влияние
study_job = study_hours_per_day × part_time_job_Yes задаёт разные наклоны для работающих и неработающих, отражая реальную модерацию ресурса времени.

4) Влияние экранного времени на учебу + нормировка
leisure_ratio = (social_media_hours + netflix_hours)/(study_hours_per_day + ε) сворачивает два слабых отрицательных фактора в одну осмысленную метрику «экраны на единицу учебы», уменьшая число сильно связанных столбцов; ε защищает от деления на ноль.

attendance_norm = attendance_percentage/100 заменяет процент на долю, упрощая масштаб и регуляризацию; исходный процент можно удалить, чтобы не держать точную линейную копию.

In [ ]:
def make_features_for_linear(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()

    # 1) Учебные часы: оставляем линейный центрированный + один квадратичный
    if 'study_hours_per_day' in df.columns:
        m = df['study_hours_per_day'].mean()
        df['study_hours_c']    = df['study_hours_per_day'] - m
        df['study_hours_c_sq'] = df['study_hours_c'] ** 2
        df = df.drop(columns=['study_hours_per_day'])

    # 2) связь сна с ментальным здоровьем
    if {'sleep_hours', 'mental_health_rating'}.issubset(df.columns):
        df['sleep_mental'] = df['sleep_hours'] * df['mental_health_rating']

    # 3) влияние подработки на время, уделяемое учебе
    if {'study_hours_per_day', 'part_time_job_Yes'}.issubset(df.columns):
        df['study_job'] = df['study_hours_per_day'] * df['part_time_job_Yes'].astype(int)

    # 4) Отношение времени развлечений к времени учебы (одна метрика вместо двух)
    if {'social_media_hours', 'netflix_hours', 'study_hours_per_day'}.issubset(df.columns):
        eps = 1e-3
        df['leisure_ratio'] = (df['social_media_hours'] + df['netflix_hours']) / (df['study_hours_per_day'] + eps)
        # По желанию: можно исключить social_media_hours и netflix_hours, чтобы избежать избыточности
        df = df.drop(columns=['social_media_hours','netflix_hours'])

    # 5) Нормировка посещаемости (замена исходного процента)
    if 'attendance_percentage' in df.columns:
        df['attendance_norm'] = df['attendance_percentage'] / 100.0
        # Для устранения точной линейной копии:
        df = df.drop(columns=['attendance_percentage'])

    return df
df_linear = make_features_for_linear(df_linear)

Разделение датасета на признаки и целевую переменную

In [ ]:
from sklearn.model_selection import train_test_split


X_linear = df_linear.drop(columns=['exam_score'])
y_linear = df_linear['exam_score']

# Разделение выборки на test/train (20/80)
X_train_linear, X_test_linear, y_train_linear, y_test_linear = train_test_split(
    X_linear, y_linear, test_size=0.2, random_state=RND_SEED
)

Данные для обучения модели нужно стандартизировать

***Определение 4: Стандартизация признаков — это метод преобразования числовых признаков так, чтобы они имели среднее значение 0 и стандартное отклонение 1. Это важный шаг в подготовке данных для моделей, чувствительных к масштабу признаков, например линейной регрессии, логистической регрессии, SVM, KNN.***

**Как это работает**

Для каждого признака $x$ вычисляется:

$$
x_\text{scaled} = \frac{x - \mu}{\sigma}
$$

где:

* $\mu$ — среднее значение признака в обучающей выборке,
* $\sigma$ — стандартное отклонение признака.

После стандартизации:

* Среднее значение нового признака ≈ 0
* Стандартное отклонение ≈ 1


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_linear_scaled = scaler.fit_transform(X_train_linear)  # вычисляет среднее и стандартное отклонение (только на train)
X_test_linear_scaled = scaler.transform(X_test_linear)  # применяет эти параметры к любым данным (train, test, новые данные)

#### 1.3.2 Random Forest Regressor

**Особенности модели:**

* Не чувствительна к масштабу признаков.
* Может обрабатывать сильные корреляции между признаками.
* Может использовать категориальные признаки, если они закодированы как числовые.


Сделаем копию датасета

In [ ]:
df_forest = df_processed.copy()

Feature engineering

In [ ]:
def make_forest_features(df: pd.DataFrame) -> pd.DataFrame:

    df_f = df.copy()

    # 1) Квадрат учебных часов — захватывает ускоряющийся рост оценки с ростом study_hours_per_day
    if 'study_hours_per_day' in df_f.columns:
        df_f['study_hours_sq'] = df_f['study_hours_per_day'] ** 2

    # 2) Отклонение сна от оптимума (8 ч) в квадрате —
    #    учитывает, что слишком мало и слишком много сна негативно влияет на оценку
    if 'sleep_hours' in df_f.columns:
        df_f['sleep_dev_sq'] = (df_f['sleep_hours'] - 8.0) ** 2

    # 3) Соотношение времени развлечений к учёбе —
    #    объединяет социальные медиа и Netflix в одну метрику влияния отвлечений
    if {'social_media_hours','netflix_hours','study_hours_per_day'}.issubset(df_f.columns):
        df_f['leisure_ratio'] = (
            df_f['social_media_hours'] + df_f['netflix_hours']
        ) / (df_f['study_hours_per_day'] + 1e-6)

    # 4) Взаимодействие сна и учёбы — отражает, что сочетание здорового сна и достаточного времени на учёбу
    if {'sleep_hours','study_hours_per_day'}.issubset(df_f.columns):
        df_f['sleep_study_inter'] = (
            df_f['sleep_hours'] * df_f['study_hours_per_day']
        )

    return df_f
df_forest = make_forest_features(df_forest)

Разделение выборки

In [ ]:
X_forest = df_forest.drop(columns=['exam_score'])
y_forest = df_forest['exam_score']

X_train_forest, X_test_forest, y_train_forest, y_test_forest = train_test_split(
    X_forest, y_forest, test_size=0.2, random_state=RND_SEED
)

#### 1.3.3 XGBoost

**Особенности модели:**

* Градиентный бустинг деревьев.
* Не чувствителен к масштабу.
* Может обрабатывать коррелированные признаки, но слишком много слабых признаков может замедлить обучение.

Сделаем копию датасета

In [ ]:
df_xgboost = df_processed.copy()

Чистим от выбросов

In [ ]:
lower_limit = df_xgboost['exam_score'].quantile(0.01)
upper_limit = df_xgboost['exam_score'].quantile(0.99)

df_xgboost = df_xgboost[(df_xgboost['exam_score'] >= lower_limit) &
                 (df_xgboost['exam_score'] <= upper_limit)]

Feature engineering

In [ ]:
def make_xgb_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Обоснованные признаки для XGBoostRegressor:
    учтено, что модель не чувствительна к масштабу,
    но слишком много слабых признаков замедляет обучение.
    """
    df_xgb = df.copy()

    # 1) Логарифм учебных часов — ловит убывающий эффект при очень долгой учёбе
    if 'study_hours_per_day' in df_xgb.columns:
        df_xgb['study_hours_log'] = np.log1p(df_xgb['study_hours_per_day'])

    # 2) Отклонение сна от оптимума (8 ч)
    if 'sleep_hours' in df_xgb.columns:
        df_xgb['sleep_dev'] = (df_xgb['sleep_hours'] - 8.0).abs()

    # 3) Соотношение развлечений к учёбе
    if {'social_media_hours','netflix_hours','study_hours_per_day'}.issubset(df_xgb.columns):
        df_xgb['leisure_ratio'] = (
            df_xgb['social_media_hours'] + df_xgb['netflix_hours']
        ) / (df_xgb['study_hours_per_day'] + 1e-6)

    # 4) Интерактивный признак посещаемость × учёба
    if {'attendance_percentage','study_hours_per_day'}.issubset(df_xgb.columns):
        df_xgb['attendance_study_inter'] = (
            df_xgb['attendance_percentage'] * df_xgb['study_hours_per_day']
        )

    return df_xgb

df_xgboost = make_xgb_features(df_xgboost)

Разделение выборки

In [ ]:
X_xgboost = df_xgboost.drop(columns=['exam_score'])
y_xgboost = df_xgboost['exam_score']

X_train_xgboost, X_test_xgboost, y_train_xgboost, y_test_xgboost = train_test_split(
    X_xgboost, y_xgboost, test_size=0.2, random_state=RND_SEED
)

## 2. Создание метрик

1. **MSE (Mean Squared Error)** – средняя квадратичная ошибка:

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

* Чем меньше, тем лучше.
* Чувствительна к выбросам (квадрат ошибки усиливает влияние больших отклонений).

2. **RMSE (Root Mean Squared Error)** – корень из MSE:

$$
\text{RMSE} = \sqrt{\text{MSE}}
$$

* В тех же единицах, что и целевая переменная.
* Легче интерпретировать.

3. **MAE (Mean Absolute Error)** – средняя абсолютная ошибка:

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
$$

* Менее чувствительна к выбросам, показывает «среднюю ошибку» в исходных единицах.

4. **R² (коэффициент детерминации)**:

$$
R^2 = 1 - \frac{\sum (y_i - \hat{y}_i)^2}{\sum (y_i - \bar{y})^2}
$$

* 1 → идеальное предсказание, 0 → модель не лучше среднего, <0 → хуже среднего.

**Как интерпретировать**

| Метрика | Как читать                    | Что значит для анализа                                                   |
| ------- | ----------------------------- | ------------------------------------------------------------------------ |
| MSE     | Чем меньше, тем точнее        | Показывает среднюю квадратичную ошибку. Выбросы сильно влияют.           |
| RMSE    | В тех же единицах, что и цель | Удобно для прямой интерпретации ошибок.                                  |
| MAE     | Средняя абсолютная ошибка     | Устойчивее к выбросам, показывает среднюю фактическую ошибку.            |
| R²      | 0–1 (или <0)                  | 1 — идеальное совпадение, 0 — предсказывает среднее, <0 — хуже среднего. |

**Пример анализа:**

* Если RMSE и MAE сильно отличаются → есть выбросы.
* Если R² близок к 1 → модель хорошо объясняет вариацию данных.
* Можно сравнивать модели: линейная, RF, XGBoost. Та, у которой меньше RMSE/MAE и выше R² — более точная.


In [ ]:
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def r2(y_true, y_pred):
    return 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))

def get_metrics(y_true, y_pred):
    return {
        'MSE': mse(y_true, y_pred),
        'RMSE': rmse(y_true, y_pred),
        'MAE': mae(y_true, y_pred),
        'R2': r2(y_true, y_pred)
    }


Прежде чем начнем обучать, создадим `DataFrame` для снятия метрик

## 3. Базовая модель линейной регрессии (аналитическое решение)

In [ ]:
# Создание линейной модели
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [ ]:
# Обучение
lr.fit(X_train_linear_scaled, y_train_linear)

In [ ]:
# Прогонка и метрики
y_pred = lr.predict(X_test_linear_scaled)
lr_metrics = get_metrics(y_test_linear, y_pred)
lr_metrics

## 4. Улучшенная версия линейной регрессии

In [ ]:
from sklearn.linear_model import SGDRegressor

### 4.1. Градиентный спуск (симуляция)

In [ ]:
# Создание линейной модели градиентного спуска
lr_gd = SGDRegressor(
    max_iter=2000,               # максимум итераций
    tol=1e-6,                    # остановка, когда улучшение < tol
    learning_rate='constant',  # тип изменения шага
    eta0=0.01,                   # стартовый шаг
    penalty='l2',                # регуляризация L2 (Ridge)
    shuffle=False,               # важный момент: не перемешиваем данные, чтобы был настоящий GD
    random_state=RND_SEED
)

In [ ]:
# Обучение
lr_gd.fit(X_train_linear_scaled, y_train_linear)

In [ ]:
# Прогонка и метрики
y_pred = lr_gd.predict(X_test_linear_scaled)
lr_gd_metrics = get_metrics(y_test_linear, y_pred)
lr_gd_metrics

### 4.2. Стохастический градиентный спуск

In [ ]:
# Инициализация модели
lr_sgd = SGDRegressor(
    max_iter=2000,                # максимум итераций
    tol=1e-6,                     # остановка, когда улучшение < tol
    learning_rate='invscaling',   # тип изменения шага
    eta0=0.01,                    # стартовый шаг
    penalty='l1',                 # регуляризация L2 (Ridge)
    random_state=RND_SEED
)

In [ ]:
# Обучение
lr_sgd.fit(X_train_linear_scaled, y_train_linear)

In [ ]:
# Прогонка и метрики
y_pred = lr_sgd.predict(X_test_linear_scaled)
lr_sgd_metrics = get_metrics(y_test_linear, y_pred)
lr_sgd_metrics

### 4.3. Линейная модель с регуляризацией Rigde (L2)

In [ ]:
# Инициализация модели
from sklearn.linear_model import Ridge
lr_ridge = Ridge(alpha=1.0)

In [ ]:
# Обучение
lr_ridge.fit(X_train_linear_scaled, y_train_linear)

In [ ]:
# Прогонка и метрики
y_pred = lr_ridge.predict(X_test_linear_scaled)
lr_ridge_metrics = get_metrics(y_test_linear, y_pred)
lr_ridge_metrics

### 4.3. Линейная модель с регуляризацией Lasso (L1)

In [ ]:
# Инициализация модели
from sklearn.linear_model import Lasso
lr_lasso = Lasso(alpha=0.01)

In [ ]:
# Обучение
lr_lasso.fit(X_train_linear_scaled, y_train_linear)

In [ ]:
# Прогонка и метрики
y_pred = lr_lasso.predict(X_test_linear_scaled)
lr_lasso_metrics = get_metrics(y_test_linear, y_pred)
lr_lasso_metrics

### 4.4. Линейная модель с регуляризацией ElasticNet (комбинация L1+L2)

In [ ]:
# Инициализация модели
from sklearn.linear_model import ElasticNet
lr_enet = ElasticNet(alpha=0.01, l1_ratio=0.5)

In [ ]:
# Обучение
lr_enet.fit(X_train_linear_scaled, y_train_linear)

In [ ]:
# Прогонка и метрики
y_pred = lr_enet.predict(X_test_linear_scaled)
lr_enet_metrics = get_metrics(y_test_linear, y_pred)
lr_enet_metrics

## 4. Случайный лес регрессор

In [ ]:
# Инициализация модели
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(
    n_estimators=200,      # количество деревьев
    max_depth=None,        # глубина деревьев
    random_state=RND_SEED,
    n_jobs=-1              # использовать все ядра процессора
)

In [ ]:
# Обучение
rf.fit(X_train_forest, y_train_forest)

In [ ]:
# Прогонка и метрики
y_pred = rf.predict(X_test_forest)
rf_metrics = get_metrics(y_test_forest, y_pred)
rf_metrics

## 5. Градиентный бустинг (XGBoost)

In [ ]:
# Инициализация модели
import xgboost
xgb = xgboost.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RND_SEED
)

In [ ]:
# Обучение
xgb.fit(X_train_xgboost, y_train_xgboost)

In [ ]:
# Прогонка и метрики
y_pred = xgb.predict(X_test_xgboost)
xgb_metrics = get_metrics(y_test_xgboost, y_pred)
xgb_metrics

## 5. Feature Importance

### 5.1 Linear Regression


In [ ]:
importance = pd.DataFrame({
    'Feature': X_train_linear.columns,
    'Coefficient': lr.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(
    x='Coefficient',
    y='Feature',
    data=importance,
    palette='viridis'
)
plt.axvline(0, color='red', linestyle='--')
plt.title('Feature Importance (Linear Regressor)')
plt.show()

### 5.2 Linear Regression (GD)


In [ ]:
importance = pd.DataFrame({
    'Feature': X_train_linear.columns,
    'Coefficient': lr_gd.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(
    x='Coefficient',
    y='Feature',
    data=importance,
    palette='viridis'
)
plt.axvline(0, color='red', linestyle='--')
plt.title('Feature Importance (GD Regressor)')
plt.show()

### 5.3 Linear Regression (SGD)


In [ ]:
importance = pd.DataFrame({
    'Feature': X_train_linear.columns,
    'Coefficient': lr_sgd.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(
    x='Coefficient',
    y='Feature',
    data=importance,
    palette='viridis'
)
plt.axvline(0, color='red', linestyle='--')
plt.title('Feature Importance (SGD Regressor)')
plt.show()

### 5.4 Linear Regression (Rigde)


In [ ]:
importance = pd.DataFrame({
    'Feature': X_train_linear.columns,
    'Coefficient': lr_ridge.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(
    x='Coefficient',
    y='Feature',
    data=importance,
    palette='viridis'
)
plt.axvline(0, color='red', linestyle='--')
plt.title('Feature Importance (LR ridge)')
plt.show()

### 5.5 Linear Regression (Lasso)


In [ ]:
importance = pd.DataFrame({
    'Feature': X_train_linear.columns,
    'Coefficient': lr_lasso.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(
    x='Coefficient',
    y='Feature',
    data=importance,
    palette='viridis'
)
plt.axvline(0, color='red', linestyle='--')
plt.title('Feature Importance (LR lasso)')
plt.show()

### 5.6 Linear Regression (ElasticNet)


In [ ]:
importance = pd.DataFrame({
    'Feature': X_train_linear.columns,
    'Coefficient': lr_enet.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(
    x='Coefficient',
    y='Feature',
    data=importance,
    palette='viridis'
)
plt.axvline(0, color='red', linestyle='--')
plt.title('Feature Importance (LR elastic_net)')
plt.show()

### 5.7 Random Forest


In [ ]:
feature_importances = pd.Series(rf.feature_importances_, index=X_forest.columns)
feature_importances = feature_importances.sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=feature_importances.values, y=feature_importances.index)
plt.title("Feature Importance (Random Forest)")
plt.show()

### 5.6 XGBoost

In [ ]:
plt.figure(figsize=(10,6))
xgboost.plot_importance(xgb, importance_type='weight', max_num_features=10)
plt.title("Feature Importance (XGBoost)")
plt.show()

## 8. Написание своих реализаций (классы)

Напишите свои классы реализации:

- LR
- LR + GD
- LR + SGD
- *Random Forest (не обязательно)
- *Gradient Boosting Regressor (не обязательно)

In [ ]:
import numpy as np

class CustomLinearRegression:
    def __init__(self, method='analytical', random_state=None):
        self.method = method
        self.weights = None
        self.bias = None
        self.cost_history = []
        self.random_state = random_state
        self._rng = np.random.default_rng(random_state)

    @staticmethod
    def _to_numpy(X, y):
        Xn = X.values if hasattr(X, 'values') else X
        yn = y.values.ravel() if hasattr(y, 'values') else np.ravel(y)
        return Xn.astype(float), yn.astype(float)

    # Аналитическое решение
    def analytical_solution(self, X, y):
        X, y = self._to_numpy(X, y)
        X_with_bias = np.c_[np.ones(X.shape[0]), X]
        try:
            XTX_inv = np.linalg.inv(X_with_bias.T @ X_with_bias)
            weights_with_bias = XTX_inv @ X_with_bias.T @ y
        except np.linalg.LinAlgError:
            weights_with_bias = np.linalg.pinv(X_with_bias) @ y
        self.bias = float(weights_with_bias[0])
        self.weights = weights_with_bias[1:].astype(float)

    # Градиентный спуск
    def gradient_descent(self, X, y, learning_rate=0.01, max_iterations=1000):
        X, y = self._to_numpy(X, y)
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features, dtype=float)
        self.bias = 0.0
        self.cost_history = []

        for i in range(max_iterations):
            y_pred = X @ self.weights + self.bias
            error = y_pred - y
            cost = np.mean(error ** 2)
            self.cost_history.append(float(cost))

            dw = (2.0 / n_samples) * (X.T @ error)
            db = (2.0 / n_samples) * np.sum(error)

            self.weights -= learning_rate * dw
            self.bias    -= learning_rate * db

            if i > 0 and abs(self.cost_history[-2] - self.cost_history[-1]) < 1e-8:
                break

    # Стохастический градиентный спуск
    def stochastic_gradient_descent(self, X, y, learning_rate=0.01, max_iterations=1000, batch_size=32):
        X, y = self._to_numpy(X, y)
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features, dtype=float)
        self.bias = 0.0
        self.cost_history = []

        for epoch in range(max_iterations):
            indices = self._rng.permutation(n_samples)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            total_cost, num_batches = 0.0, 0

            for i in range(0, n_samples, batch_size):
                end_idx = min(i + batch_size, n_samples)
                X_batch = X_shuffled[i:end_idx]
                y_batch = y_shuffled[i:end_idx]

                y_pred = X_batch @ self.weights + self.bias
                error = y_pred - y_batch
                cost = np.mean(error ** 2)

                m = X_batch.shape[0]
                dw = (2.0 / m) * (X_batch.T @ error)
                db = (2.0 / m) * np.sum(error)

                self.weights -= learning_rate * dw
                self.bias    -= learning_rate * db

                total_cost += float(cost)
                num_batches += 1

            self.cost_history.append(total_cost / max(num_batches, 1))

            if epoch > 0 and abs(self.cost_history[-2] - self.cost_history[-1]) < 1e-8:
                break

    def fit(self, X, y, **kwargs):
        if self.method == 'analytical':
            self.analytical_solution(X, y)
        elif self.method == 'gradient_descent':
            self.gradient_descent(X, y, **kwargs)
        elif self.method == 'sgd':
            self.stochastic_gradient_descent(X, y, **kwargs)
        else:
            raise ValueError("Неподдерживаемый метод")
        return self

    def predict(self, X):
        if self.weights is None:
            raise ValueError("Сначала вызовите fit()")
        Xn = X.values if hasattr(X, 'values') else X
        Xn = Xn.astype(float)
        return Xn @ self.weights + self.bias

    def score(self, X, y):
        y_true = y.values.ravel() if hasattr(y, 'values') else np.ravel(y)
        y_pred = self.predict(X)
        ss_res = np.sum((y_true - y_pred) ** 2)
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
        return 1.0 - ss_res / ss_tot



In [ ]:
custom_lr = CustomLinearRegression("analytical")
custom_lr.fit(X_train_linear_scaled, y_train_linear)


In [ ]:
y_pred = custom_lr.predict(X_test_linear_scaled)
custom_lr_metrics = get_metrics(y_test_linear, y_pred);
custom_lr_metrics


In [ ]:
custom_lr = CustomLinearRegression("gradient_descent")
custom_lr.fit(X_train_linear_scaled, y_train_linear)

In [ ]:
y_pred = custom_lr.predict(X_test_linear_scaled)
custom_lr_gd_metrics = get_metrics(y_test_linear, y_pred);
custom_lr_gd_metrics

In [ ]:
custom_lr = CustomLinearRegression("sgd")
custom_lr.fit(X_train_linear_scaled, y_train_linear)


In [ ]:
y_pred = custom_lr.predict(X_test_linear_scaled)
custom_lr_sgd_metrics = get_metrics(y_test_linear, y_pred);
custom_lr_sgd_metrics

## 9. Итоги

Что сделать?

1. Сгрупировать все метрики, и выяснить, какой метод сработал лучше всего и почему?
2. Ответить на вопросы:

    1. Что такое регрессия и чем она отличается от классификации?
    2. Какова целевая переменная в задаче регрессии?
    3. Зачем нужно масштабирование признаков перед обучением линейной регрессии?
    4. Что означает коэффициент признака в линейной регрессии?
    5. Что такое MSE, RMSE, MAE и R², и чем они отличаются?
    6. В чем разница между Ridge и Lasso регуляризацией?
    7. Почему деревья решений и Random Forest не требуют стандартизации признаков?
    8. Что такое мультиколлинеарность и почему она мешает линейной регрессии?
    9. Как можно уменьшить влияние выбросов на линейную регрессию?
    10. Какие гиперпараметры наиболее важны для Random Forest Regressor?
    11. Какие гиперпараметры наиболее важны для XGBoost в задаче регрессии?
    12. Что значит глубина дерева (max\_depth) и как она влияет на модель?
    13. Зачем нужен `learning_rate` в градиентном бустинге?
    14. Как можно оценить важность признаков (feature importance) в линейной регрессии, случайном лесу и XGBoost?
    15. Почему XGBoost часто работает лучше, чем Random Forest, на структурированных данных?
    16. Что такое переобучение и как его можно выявить на графике обучения?
    17. Как работает метод ансамблирования в Random Forest (bagging)?
    18. В чем отличие бустинга от бэггинга?
    19. Какие способы feature engineering можно применить к вашему датасету?
    20. Как использовать кросс-валидацию для подбора гиперпараметров моделей регрессии?


#### 9.1. Группировка метрик

In [ ]:
# code here

method2metrics = {
    'LinearRegression':              lr_metrics,
    'Gradient descend':              lr_gd_metrics,
    'SGDRegressor':                  lr_sgd_metrics,
    'Ridge':                         lr_ridge_metrics,
    'Lasso':                         lr_lasso_metrics,
    'ElasticNet':                    lr_enet_metrics,
    'RandomForestRegressor':         rf_metrics,
    'XGBRegressor':                  xgb_metrics,

    'Custom LR (analytical)':        custom_lr_metrics,
    'Custom LR + GD':                custom_lr_gd_metrics,
    'Custom LR + SGD':               custom_lr_sgd_metrics,
}

# 2) Нормализуем типы и оставляем ключевые метрики
norm = {}
for method, md in method2metrics.items():
    if md is None:
        continue
    norm[method] = {
        k: float(v) for k, v in md.items()
        if k in ('MSE', 'RMSE', 'MAE', 'R2')
    }

import pandas as pd
df = pd.DataFrame.from_dict(norm, orient='index')
desired = ['MSE', 'RMSE', 'MAE', 'R2']
cols = [c for c in desired if c in df.columns] + [c for c in df.columns if c not in desired]
df = df[cols].sort_values('MSE')
df.index.name = 'method'


df.round(6)

Вывод: данные ведут себя почти линейно, поэтому регуляризованные линейные модели закономерно обгоняют деревья, а отсутствие тщательного тюнинга ухудшило результаты XGBoost и случайного леса. Небольшой выигрыш ElasticNet объясним совместной L1+L2 регуляризацией, которая одновременно стабилизирует коэффициенты при мультиколлинеарности и делает мягкий отбор признаков.

Дерево - кусочно‑постоянная аппроксимация, и чтобы точно повторить почти прямую зависимость, нужны глубокие деревья. при типовых настройках получается недообучение относительно простой линейной функции.



#### 9.2. Ответы на вопросы

1. Что такое регрессия и чем она отличается от классификации?

Регрессия — это задача машинного обучения с учителем для предсказания непрерывных числовых значений. В основе регрессионного анализа лежит построение функциональной зависимости между независимыми переменными (признаками) и зависимой переменной (целевой). Классификация же предсказывает категориальные переменные — принадлежность объекта к определенному классу.

Ключевые отличия: в регрессии целевая переменная может принимать бесконечное множество значений (цены домов, температура, экзаменационные баллы как в вашей ЛР), в классификации — конечное множество дискретных категорий (spam/not spam, диагнозы болезней). Метрики также различаются: для регрессии используют MSE, RMSE, MAE, R², для классификации — accuracy, precision, recall, F1-score.

2. Какова целевая переменная в задаче регрессии?

Целевая переменная в регрессии — это количественная переменная, которую мы пытаемся предсказать. Она должна быть измеримой и принимать числовые значения из непрерывного или дискретного множества. В нашей лабораторной работе целевая переменная — exam_score.

Другие примеры: стоимость недвижимости, температура воздуха, объем продаж, доходы компании, количество клиентов. Важно, чтобы целевая переменная была численно осмысленной — разность между значениями должна иметь интерпретацию (разница между 80 и 70 баллами такая же, как между 60 и 50).

3. Зачем нужно масштабирование признаков перед обучением линейной регрессии?

Масштабирование критично для линейных моделей, поскольку признаки имеют разные единицы измерения и диапазоны значений. В нашем датасете: age (17-24), study_hours_per_day (0-8.3), attendance_percentage (56-100). Без масштабирования признаки с большими значениями будут доминировать в процессе оптимизации.

Проблемы без масштабирования: градиентный спуск сходится медленно или вообще не сходится, коэффициенты становятся несопоставимыми по величине, численная неустойчивость алгоритмов. StandardScaler приводит все признаки к стандартному нормальному распределению (μ=0, σ=1), что обеспечивает равные условия для всех переменных в процессе оптимизации и правильную интерпретацию коэффициентов.

4. Что означает коэффициент признака в линейной регрессии?

Коэффициент показывает изменение целевой переменной при изменении соответствующего признака на одну единицу, при условии, что все остальные признаки остаются неизменными. В нашей ЛР коэффициент для study_hours_c равен ~12.5, что означает: каждый дополнительный час учебы увеличивает экзаменационный балл в среднем на 12.5 пунктов.

5. Что такое MSE, RMSE, MAE и R², и чем они отличаются?

MSE (Mean Squared Error) = (1/n) × Σ(y_true - y_pred)² — средняя квадратичная ошибка. Сильно штрафует большие ошибки из-за возведения в квадрат, единицы измерения — квадрат целевой переменной.

RMSE (Root Mean Squared Error) = √MSE — корень из MSE. Возвращает ошибку в исходных единицах (баллы), легко интерпретируется. RMSE=5.26 означает типичную ошибку ±5.26 балла.

MAE (Mean Absolute Error) = (1/n) × Σ|y_true - y_pred| — средняя абсолютная ошибка. Устойчива к выбросам, все ошибки весят одинаково.

R² (коэффициент детерминации) = 1 - SS_res/SS_tot — доля объясненной дисперсии. R²=0.891 в нашей ЛР означает, что модель объясняет 89.1% вариации экзаменационных баллов.

6. В чем разница между Ridge и Lasso регуляризацией?

Ridge (L2-регуляризация) добавляет к функции потерь штраф α×Σβᵢ² (сумма квадратов коэффициентов). Уменьшает коэффициенты к нулю, но никогда не обнуляет их полностью. Эффективна при мультиколлинеарности — распределяет веса между коррелирующими признаками.

Lasso (L1-регуляризация) добавляет штраф α×Σ|βᵢ| (сумма абсолютных значений). Может полностью обнулять коэффициенты, выполняя автоматический отбор признаков. Создает разреженные модели, где многие коэффициенты равны нулю.

7. Почему деревья решений и Random Forest не требуют стандартизации признаков?

Деревья принимают решения на основе пороговых значений и упорядочивания, а не расстояний или линейных комбинаций. Алгоритм выбирает лучшее разбиение по принципу "если study_hours > 4, то...", где важен только относительный порядок значений, а не их абсолютная величина.

Инвариантность к монотонным преобразованиям: умножение признака на константу или добавление константы не меняет результат разбиения. Дерево одинаково разделит данные по признаку со значениями. Random Forest наследует это свойство, поскольку состоит из деревьев. Это существенное преимущество tree-based моделей — они работают с разнородными признаками без предварительной обработки.

8. Что такое мультиколлинеарность и почему она мешает линейной регрессии?

Мультиколлинеарность — сильная корреляция между предикторами, когда один признак можно выразить через линейную комбинацию других.

Проблемы для линейной регрессии: коэффициенты становятся неустойчивыми — малые изменения в данных приводят к резким изменениям весов; затрудняется интерпретация — неясно, какой признак действительно важен; численная неустойчивость — матрица X^T×X становится плохо обусловленной или даже вырожденной.

9. Как можно уменьшить влияние выбросов на линейную регрессию?

Методы обработки выбросов: 1) Удаление — исключение объектов за пределами 1-99 перцентилей (как в нашей ЛР); 2) Обрезание (clipping) — ограничение экстремальных значений; 3) Трансформация данных — логарифмирование, Box-Cox преобразование для снижения асимметрии.

10. Какие гиперпараметры наиболее важны для Random Forest Regressor?

n_estimators — количество деревьев в лесу. В вашей ЛР используется 200. Больше деревьев = выше качество, но медленнее обучение. Обычно 100-1000, после определенного значения улучшение незначительно.

max_depth — максимальная глубина каждого дерева. Контролирует сложность: глубокие деревья могут переобучаться, мелкие — недообучаться. None (по умолчанию) означает деревья растут до чистых листьев.

min_samples_split — минимальное количество образцов для разбиения узла (по умолчанию 2). min_samples_leaf — минимум образцов в листе (по умолчанию 1). Эти параметры контролируют переобучение.

max_features — количество признаков для рассмотрения в каждом разбиении. По умолчанию для регрессии — все признаки, но √n_features часто работает лучше. random_state — воспроизводимость результатов.

11. Какие гиперпараметры наиболее важны для XGBoost в задаче регрессии?

n_estimators — количество деревьев (в вашей ЛР 500). Больше итераций = лучше качество, но выше риск переобучения. learning_rate (eta) — скорость обучения (0.05 в ЛР). Малые значения требуют больше деревьев, но дают стабильное обучение.

max_depth — глубина деревьев (4 в ЛР). XGBoost обычно использует неглубокие деревья (3-6), в отличие от Random Forest. subsample — доля образцов для обучения каждого дерева (0.8). Уменьшает переобучение через случайность.

colsample_bytree — доля признаков для каждого дерева (0.8). reg_alpha (L1) и reg_lambda (L2) — регуляризация для предотвращения переобучения. early_stopping_rounds — остановка при отсутствии улучшения на валидации.

Стратегия настройки: начать с learning_rate=0.1, подобрать n_estimators, затем уменьшить learning_rate и увеличить n_estimators.

12. Что значит глубина дерева (max_depth) и как она влияет на модель?

max_depth определяет максимальное количество уровней от корня до самого глубокого листа. Это ключевой параметр сложности модели — более глубокие деревья могут выучить более сложные паттерны, но склонны к переобучению.

Влияние на модель: мелкие деревья (depth=2-3) создают простые правила, могут недообучаться; глубокие деревья (depth>10) запоминают специфичные комбинации признаков из обучающей выборки, плохо генерализуют.

В Random Forest: каждое дерево может быть глубоким (часто без ограничений), поскольку усреднение по многим деревьям снижает переобучение. В XGBoost: обычно используются неглубокие деревья (3-6 уровней), так как бустинг итеративно улучшает предсказания.

Выбор оптимальной глубины: кросс-валидация, мониторинг метрик на валидационной выборке.

13. Зачем нужен learning_rate в градиентном бустинге?

Learning_rate (скорость обучения) контролирует вклад каждого нового дерева в финальное предсказание. Формула обновления: F_new = F_old + learning_rate × h(x), где h(x) — предсказание нового дерева.

При высоком learning_rate (близко к 1): быстрое обучение, но высокий риск переобучения и нестабильность. При низком learning_rate (0.01-0.1): медленное, но стабильное обучение, лучшая генерализация, требует больше деревьев.

В нашей ЛР: learning_rate=0.05 — консервативное значение, обеспечивающее баланс между скоростью и качеством. Trade-off: низкий learning_rate + много деревьев часто дает лучшие результаты.

Адаптивные стратегии: начать с высокого значения, постепенно уменьшать; использовать early stopping для автоматической остановки.

14. Как можно оценить важность признаков (feature importance) в линейной регрессии, случайном лесу и XGBoost?

Линейная регрессия: важность определяется абсолютными значениями коэффициентов после стандартизации признаков.

Random Forest: использует MDI (Mean Decrease in Impurity) — измеряет, насколько каждый признак уменьшает неопределенность при разбиениях по всем деревьям. В вашей ЛР sleep_study_inter доминирует с важностью ~0.35.

XGBoost: несколько типов важности — weight (частота использования признака), gain (среднее улучшение качества), cover (относительное количество наблюдений). Gain часто наиболее информативен.

Интерпретация: важность показывает относительный вклад, но не причинно-следственную связь. Коррелирующие признаки могут «делить» важность между собой.

15. Почему XGBoost часто работает лучше, чем Random Forest, на структурированных данных?

Градиентный бустинг vs Бэггинг: XGBoost последовательно исправляет ошибки, каждое дерево фокусируется на сложных случаях. Random Forest усредняет независимые предсказания, что может «размывать» сигнал.

Оптимизация: XGBoost использует продвинутые техники оптимизации — второй порядок градиентов (Newton-Raphson), встроенная регуляризация, обработка пропущенных значений. Random Forest полагается на простое усреднение.

Гибкость: XGBoost предоставляет множество гиперпараметров для тонкой настройки — различные функции потерь, схемы регуляризации, стратегии выборки. Обработка дисбаланса: лучше справляется с несбалансированными данными через веса классов.

16. Что такое переобучение и как его можно выявить на графике обучения?

Переобучение (overfitting) — модель «запоминает» обучающие данные вместо изучения общих закономерностей. Отлично работает на train set, плохо на новых данных.

Признаки на learning curves: train score продолжает расти, validation score стагнирует или падает — классический признак переобучения. Большой разрыв между train и validation метриками. Validation loss растет после определенной точки, несмотря на снижение train loss.

В нашей ЛР: хорошие результаты (R²=0.891) без большого разрыва между train/test указывают на отсутствие серьезного переобучения благодаря правильной регуляризации и feature engineering.

Методы борьбы: регуляризация (Ridge/Lasso), early stopping, dropout, кросс-валидация, увеличение данных, уменьшение сложности модели.

17. Как работает метод ансамблирования в Random Forest (bagging)?

Bagging (Bootstrap Aggregating) — каждое дерево обучается на случайной подвыборке с возвращением (bootstrap sample) размером равным исходной выборке. Некоторые объекты попадают несколько раз, другие не попадают вовсе (~37% остаются out-of-bag).

Дополнительная случайность: в каждом узле каждого дерева рассматривается случайное подмножество признаков (обычно √p для классификации, p/3 для регрессии). Это снижает корреляцию между деревьями.

Агрегация результатов: для регрессии — среднее арифметическое предсказаний всех деревьев, для классификации — мажоритарное голосование. В вашей ЛР 200 деревьев усредняют свои предсказания экзаменационных баллов.

Преимущества bagging: снижение дисперсии модели, защита от переобучения, параллелизуемость обучения, out-of-bag оценка качества без отдельной валидационной выборки.

18. В чем отличие бустинга от бэггинга?

Бэггинг (Random Forest): деревья обучаются параллельно и независимо на разных подвыборках. Каждое дерево имеет равный вес в финальном решении. Цель — снизить дисперсию через усреднение.

Бустинг (XGBoost): деревья обучаются последовательно, каждое следующее исправляет ошибки предыдущих. Более поздние деревья фокусируются на сложных случаях. Цель — снизить bias через итеративное улучшение.

Веса объектов: в бэггинге все объекты равноважны, в бустинге объекты с большими ошибками получают больший вес в следующей итерации.

Риски: бэггинг редко переобучается благодаря усреднению, бустинг склонен к переобучению при избыточных итерациях.

Performance: бустинг часто достигает лучшего качества на train set, но требует более аккуратной настройки для хорошей генерализации.

19. Какие способы feature engineering можно применить к вашему датасету?
Взаимодействие признаков: sleep_hours × study_hours, attendance × study_hours, mental_health × sleep_hours — выявляют синергетические эффекты.

Полиномиальные признаки: study_hours², sleep_hours² — моделируют нелинейности (например, оптимальное количество сна). Пропорции: leisure_time/study_time, social_media/total_screen_time — относительные метрики важнее абсолютных.

Временные паттерны: если есть временные данные — дни недели, сезонность, тренды. Биннинг: группировка непрерывных признаков в категории — "мало сна" (< 6h), "норма" (6-8h), "много" (> 8h).

Доменные знания: создание композитных индексов — "здоровый образ жизни" = exercise + sleep + diet, "академическая активность" = study_hours + attendance.

Кодирование категорий: target encoding для высококардинальных категорий, frequency encoding, label encoding с учетом порядка.

20. Как использовать кросс-валидацию для подбора гиперпараметров моделей регрессии?

K-Fold Cross-Validation: данные разделяются на K фолдов (обычно 5 или 10), модель обучается на K-1 фолдах, тестируется на оставшемся. Процедура повторяется K раз, результаты усредняются.

GridSearchCV: перебор всех комбинаций гиперпараметров из заданной сетки. Для каждой комбинации выполняется кросс-валидация. Выбирается комбинация с лучшей средней метрикой.

RandomizedSearchCV: случайная выборка из пространства гиперпараметров. Эффективнее GridSearch при большом количестве параметров.

Стратегии для регрессии:

Linear models: подбор alpha для регуляризации (10^-4 до 10^2)

Random Forest: n_estimators (100-1000), max_depth (None, 10, 20), min_samples_split (2-10)

XGBoost: learning_rate (0.01-0.3), max_depth (3-10), n_estimators (100-2000)

Метрики для оценки: R², RMSE, MAE в зависимости от задачи. Nested CV: внешняя кросс-валидация для оценки качества, внутренняя — для подбора гиперпараметров, избегает переобучения на валидации.